In [1]:
import numpy as np
import pandas as pd
import hvplot.pandas

In [2]:
# Set the random seed for reproducibility
from numpy.random import seed
seed(1)
from tensorflow import random
random.set_seed(2)

In [24]:
# Load the fear and greed sentiment data for Bitcoin
df = pd.read_csv('btc_sentiment.csv', index_col="date", parse_dates=True, dayfirst=True)
df = df.drop(columns="fng_classification")
df.head()

,fng_value
date,
2019-07-29,19
2019-07-28,16
2019-07-27,47
2019-07-26,24
2019-07-25,42


In [6]:
# Load the historical closing prices for Bitcoin
df2 = pd.read_csv('btc_historic.csv', index_col="Date", parse_dates=True)['Close']
df2 = df2.sort_index()
df2.tail()

Date
2019-07-25    9882.429688
2019-07-26    9847.450195
2019-07-27    9478.320313
2019-07-28    9531.769531
2019-07-29    9529.889648
Name: Close, dtype: float64

In [7]:
# Join the data into a single DataFrame
df = df.join(df2, how="inner")
df.tail()

,fng_value,Close
date,,
2019-07-25,42,9882.429688
2019-07-26,24,9847.450195
2019-07-27,47,9478.320313
2019-07-28,16,9531.769531
2019-07-29,19,9529.889648


In [8]:
df.head()

,fng_value,Close
date,,
2018-02-01,30,9114.719727
2018-02-02,15,8870.820313
2018-02-03,40,9251.269531
2018-02-04,24,8218.049805
2018-02-05,11,6937.080078


This function accepts the column number for the features (X) and the target (y), it chunks the data up with a rolling window of Xt-n to predict Xt and returns a numpy array of X any y

In [9]:
def window_data(df, window, feature_col_number, target_col_number):
    X = []
    y = []
    for i in range(len(df) - window - 1):
        features = df.iloc[i:(i + window), feature_col_number]
        target = df.iloc[(i + window), target_col_number]
        X.append(features)
        y.append(target)
    return np.array(X), np.array(y).reshape(-1, 1)

Predict Closing Prices using a 10 day window of previous closing prices

In [10]:
window_size = 1

# Column index 0 is the 'fng_value' column
# Column index 1 is the `Close` column
feature_column = 1
target_column = 1
X, y = window_data(df, window_size, feature_column, target_column)

Use 70% of the data for training and the remaineder for testing

In [11]:
# Use 70% of the data for training and the remaineder for testing
split = int(0.7 * len(X))
X_train = X[: split]
X_test = X[split:]
y_train = y[: split]
y_test = y[split:]

Use the MinMaxScaler to scale data between 0 and 1.

In [12]:
from sklearn.preprocessing import MinMaxScaler

scaler = MinMaxScaler()
scaler.fit(X)
X_train = scaler.transform(X_train)
X_test = scaler.transform(X_test)
scaler.fit(y)
y_train = scaler.transform(y_train)
y_test = scaler.transform(y_test)

Reshape the features for the model

In [13]:
X_train = X_train.reshape((X_train.shape[0], X_train.shape[1], 1))
X_test = X_test.reshape((X_test.shape[0], X_test.shape[1], 1))
print (f"X_train sample values:\n{X_train[:5]} \n")
print (f"X_test sample values:\n{X_test[:5]}")

X_train sample values:
[[[0.60761794]]

 [[0.58242373]]

 [[0.62172321]]

 [[0.51499412]]

 [[0.38267307]]] 

X_test sample values:
[[[0.03974167]]

 [[0.04528668]]

 [[0.04528668]]

 [[0.07024855]]

 [[0.07145402]]]


### Build and Train the LSTM RNN

In [14]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, Dense, Dropout

Build the LSTM model. 

In [15]:
model = Sequential()

number_units = 5
dropout_fraction = 0.2

# Layer 1
model.add(LSTM(
    units=number_units,
    return_sequences=True,
    input_shape=(X_train.shape[1], 1))
    )
model.add(Dropout(dropout_fraction))
# Layer 2
model.add(LSTM(units=number_units, return_sequences=True))
model.add(Dropout(dropout_fraction))

# Layer 3
model.add(LSTM(units=number_units))
model.add(Dropout(dropout_fraction))
# Output layer
model.add(Dense(1))

Compile the model

In [16]:
model.compile(optimizer="adam", loss="mean_squared_error")

Summarize the model

In [17]:
model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 lstm (LSTM)                 (None, 1, 5)              140       
                                                                 
 dropout (Dropout)           (None, 1, 5)              0         
                                                                 
 lstm_1 (LSTM)               (None, 1, 5)              220       
                                                                 
 dropout_1 (Dropout)         (None, 1, 5)              0         
                                                                 
 lstm_2 (LSTM)               (None, 5)                 220       
                                                                 
 dropout_2 (Dropout)         (None, 5)                 0         
                                                                 
 dense (Dense)               (None, 1)                 6

Use at least 10 epochs to train the model

In [18]:
model.fit(X_train, y_train, epochs=10, shuffle=False, batch_size=1, verbose=1)

Epoch 1/10
378/378 [==============================] - 3s 850us/step - loss: 0.0846
Epoch 2/10
378/378 [==============================] - 0s 828us/step - loss: 0.0418
Epoch 3/10
378/378 [==============================] - 0s 766us/step - loss: 0.0393
Epoch 4/10
378/378 [==============================] - 0s 794us/step - loss: 0.0313
Epoch 5/10
378/378 [==============================] - 0s 761us/step - loss: 0.0254
Epoch 6/10
378/378 [==============================] - 0s 808us/step - loss: 0.0159
Epoch 7/10
378/378 [==============================] - 0s 771us/step - loss: 0.0118
Epoch 8/10
378/378 [==============================] - 0s 753us/step - loss: 0.0111
Epoch 9/10
378/378 [==============================] - 0s 993us/step - loss: 0.0088
Epoch 10/10
378/378 [==============================] - 0s 757us/step - loss: 0.0085


### Model Performance

In [19]:
# Evaluate the model
model.evaluate(X_test, y_test)

6/6 [==============================] - 1s 5ms/step - loss: 0.0160


0.016011608764529228

In [20]:
# Make some predictions
predicted = model.predict(X_test)

6/6 [==============================] - 1s 762us/step


In [21]:
# Recover the original prices instead of the scaled version
predicted_prices = scaler.inverse_transform(predicted)
real_prices =scaler.inverse_transform(y_test.reshape(-1, 1))

In [22]:
# Create a DataFrame of Real and Predicted values
stocks = pd.DataFrame({
    "Real": real_prices.ravel(),
    "Predicted": predicted_prices.ravel()
}, index = df.index[-len(real_prices): ]) 
stocks.head()

,Real,Predicted
date,,
2019-02-17,3670.919922,3736.445557
2019-02-18,3670.919922,3757.098877
2019-02-19,3912.570068,3757.098877
2019-02-20,3924.239990,3853.832031
2019-02-21,3974.050049,3858.661621


In [23]:
# Plot the real vs predicted values as a line chart
stocks.hvplot(title='closing prices for a 1 day window')

:NdOverlay   [Variable]
   :Curve   [date]   (value)